In [3]:
import json
import loader as Loader
import parser as Parser
from bs4 import BeautifulSoup

In [2]:
loader = Loader.Loader()
loader.crawler('https://it.pracuj.pl/praca', 'pl')

'pl_1707575579327143.html'

In [3]:
with open('./temp_data/pl_1707574037586477.html', 'r') as file:
    soup = BeautifulSoup(file, 'html.parser')

In [9]:
data_string = soup.find('script', id='__NEXT_DATA__').string
data_dict = json.loads(data_string)
with open('data.json', 'w') as file:
    json.dump(data_dict, file, indent=4)

In [5]:
parser = Parser.Parser(file_path='./temp_data/pl_1707575579327143.html', base_file_name='page_1')
parser.parse()